# Lichess game dataset
## General Info
As a passionate chess player, I am always interested in learning from how others play the game. When I saw that [Mitchell J](https://www.kaggle.com/datasnaek) uploaded a data set of just over 20,000 games collected from a selection of users on the site Lichess.org, I knew that I had to dig into it.

### Original Dataset Features
- Game ID
- Rated (T/F)
- Start Time
- End Time
- Number of Turns
- Game Status
- Winner
- Time Increment
- White Player ID
- White Player Rating
- Black Player ID
- Black Player Rating
- All Match Moves in Standard Chess Notation
- Opening Eco, the standard code for an opening
- Opening Name
- Opening Ply (Number of moves in the opening phase)

A note from [Mitchell](https://www.kaggle.com/datasnaek) on how to update the dataset:
> I collected this data using the Lichess API, which enables collection of any given users game history. The difficult part was collecting usernames to use, however the API also enables dumping of all users in a Lichess team. There are several teams on Lichess with over 1,500 players, so this proved an effective way to get users to collect games from.

### Questions of Interest
- What is the percentage of rated games?
- What is the typical length of a game on Lichess? Do time controls typically change the pace of the game? (Where in the total allowed time do games typically end?)
- For how many turns do games typically last?
- What is the typical rating differential between players? Are wins correlated to higher rating? By how much?
- Which are the most effective openings by win percentage? Least effective? Which openings are used the most at different ratings? How far do people stay in opening books, and for which openings?
- Who are the most prolific players, and what is their rating?
- Do the number of games a user plays correlate to their rating?

## Import and clean the dataset

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.2f}'.format

original_data = pd.read_csv('../data/lichess_games.csv')
original_data.head()

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1504210000000.00,1504210000000.00,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1504130000000.00,1504130000000.00,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1504130000000.00,1504130000000.00,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1504110000000.00,1504110000000.00,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1504030000000.00,1504030000000.00,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [23]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              20058 non-null  object 
 1   rated           20058 non-null  bool   
 2   created_at      20058 non-null  float64
 3   last_move_at    20058 non-null  float64
 4   turns           20058 non-null  int64  
 5   victory_status  20058 non-null  object 
 6   winner          20058 non-null  object 
 7   increment_code  20058 non-null  object 
 8   white_id        20058 non-null  object 
 9   white_rating    20058 non-null  int64  
 10  black_id        20058 non-null  object 
 11  black_rating    20058 non-null  int64  
 12  moves           20058 non-null  object 
 13  opening_eco     20058 non-null  object 
 14  opening_name    20058 non-null  object 
 15  opening_ply     20058 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(9)
memory usage: 2.3+ MB


In [24]:
original_data.describe(include='all')

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
count,20058,20058,20058.00,20058.00,20058.00,20058,20058,20058,20058,20058.00,20058,20058.00,20058,20058,20058,20058.00
unique,19113,2,NaN,NaN,NaN,4,3,400,9438,NaN,9331,NaN,18920,365,1477,NaN
top,XRuQPSzH,True,NaN,NaN,NaN,resign,white,10+0,taranga,NaN,taranga,NaN,e4 e5,A00,Van't Kruijs Opening,NaN
freq,5,16155,NaN,NaN,NaN,11147,10001,7721,72,NaN,82,NaN,27,1007,368,NaN
mean,NaN,NaN,1483616852629.09,1483617722336.14,60.47,NaN,NaN,NaN,NaN,1596.63,NaN,1588.83,NaN,NaN,NaN,4.82
std,NaN,NaN,28501509421.00,28501400588.89,33.57,NaN,NaN,NaN,NaN,291.25,NaN,291.04,NaN,NaN,NaN,2.80
min,NaN,NaN,1376771633173.00,1376771863841.00,1.00,NaN,NaN,NaN,NaN,784.00,NaN,789.00,NaN,NaN,NaN,1.00
25%,NaN,NaN,1477547500000.00,1477547500000.00,37.00,NaN,NaN,NaN,NaN,1398.00,NaN,1391.00,NaN,NaN,NaN,3.00
50%,NaN,NaN,1496010000000.00,1496010000000.00,55.00,NaN,NaN,NaN,NaN,1567.00,NaN,1562.00,NaN,NaN,NaN,4.00
75%,NaN,NaN,1503170000000.00,1503170000000.00,79.00,NaN,NaN,NaN,NaN,1793.00,NaN,1784.00,NaN,NaN,NaN,6.00


In [25]:
original_data.isnull().sum()

id                0
rated             0
created_at        0
last_move_at      0
turns             0
victory_status    0
winner            0
increment_code    0
white_id          0
white_rating      0
black_id          0
black_rating      0
moves             0
opening_eco       0
opening_name      0
opening_ply       0
dtype: int64

In [26]:
# Create helper columns
working_data = original_data.assign(game_length = lambda x:x.last_move_at.sub(x.created_at),
                                   rating_diff = lambda x: x.white_rating.sub(x.black_rating))
working_data.describe(include='all')

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply,game_length,rating_diff
count,20058,20058,20058.00,20058.00,20058.00,20058,20058,20058,20058,20058.00,20058,20058.00,20058,20058,20058,20058.00,20058.00,20058.00
unique,19113,2,NaN,NaN,NaN,4,3,400,9438,NaN,9331,NaN,18920,365,1477,NaN,NaN,NaN
top,XRuQPSzH,True,NaN,NaN,NaN,resign,white,10+0,taranga,NaN,taranga,NaN,e4 e5,A00,Van't Kruijs Opening,NaN,NaN,NaN
freq,5,16155,NaN,NaN,NaN,11147,10001,7721,72,NaN,82,NaN,27,1007,368,NaN,NaN,NaN
mean,NaN,NaN,1483616852629.09,1483617722336.14,60.47,NaN,NaN,NaN,NaN,1596.63,NaN,1588.83,NaN,NaN,NaN,4.82,869707.05,7.80
std,NaN,NaN,28501509421.00,28501400588.89,33.57,NaN,NaN,NaN,NaN,291.25,NaN,291.04,NaN,NaN,NaN,2.80,4814222.09,249.04
min,NaN,NaN,1376771633173.00,1376771863841.00,1.00,NaN,NaN,NaN,NaN,784.00,NaN,789.00,NaN,NaN,NaN,1.00,0.00,-1605.00
25%,NaN,NaN,1477547500000.00,1477547500000.00,37.00,NaN,NaN,NaN,NaN,1398.00,NaN,1391.00,NaN,NaN,NaN,3.00,0.00,-108.00
50%,NaN,NaN,1496010000000.00,1496010000000.00,55.00,NaN,NaN,NaN,NaN,1567.00,NaN,1562.00,NaN,NaN,NaN,4.00,240612.00,3.00
75%,NaN,NaN,1503170000000.00,1503170000000.00,79.00,NaN,NaN,NaN,NaN,1793.00,NaN,1784.00,NaN,NaN,NaN,6.00,793171.25,122.00


In [30]:
working_data.created_at.describe()
# Is this a UNIX timestamp? I'll come back to this.

count           20058.00
mean    1483616852629.09
std       28501509421.00
min     1376771633173.00
25%     1477547500000.00
50%     1496010000000.00
75%     1503170000000.00
max     1504493143790.00
Name: created_at, dtype: float64

## Explore the variables
### Game ID

In [31]:
working_data.id.nunique()

19113

In [33]:
# There should be a unique ID for all games. Let's check for duplicates with `id` as a subset.
working_data.duplicated(subset='id').sum()

945

In [38]:
duplicated_games = working_data[working_data.duplicated(subset='id', keep=False)]
duplicated_games.sort_values(by='id').head()

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply,game_length,rating_diff
11279,061LCiAb,True,1396112980274.00,1396113867708.00,66,draw,draw,10+15,faraon7,1630,vladimir-kramnik-1,1584,e4 e6 Nf3 d5 exd5 exd5 Bb5+ Bd7 Bxd7+ Qxd7 O-O...,C00,French Defense: Knight Variation,3,887434.00,46
10570,061LCiAb,True,1396112980274.00,1396113867708.00,66,draw,draw,10+15,faraon7,1630,vladimir-kramnik-1,1584,e4 e6 Nf3 d5 exd5 exd5 Bb5+ Bd7 Bxd7+ Qxd7 O-O...,C00,French Defense: Knight Variation,3,887434.00,46
12208,079kHDqh,True,1504113218482.00,1504113454255.00,32,resign,black,10+0,wiggleitjiggleit,1448,penguin_lo,1436,d4 d5 c4 Bf5 Nf3 e6 e3 Nc6 cxd5 Qxd5 Bd3 Bb4+ ...,D02,Queen's Gambit Refused: Baltic Defense,4,235773.00,12
720,079kHDqh,True,1504110000000.00,1504110000000.00,32,resign,black,10+0,wiggleitjiggleit,1448,penguin_lo,1436,d4 d5 c4 Bf5 Nf3 e6 e3 Nc6 cxd5 Qxd5 Bd3 Bb4+ ...,D02,Queen's Gambit Refused: Baltic Defense,4,0.00,12
3882,07e0uVvn,True,1491950000000.00,1491960000000.00,35,mate,white,30+0,chessguy9001,1259,helykopter33,1319,e4 e5 Nf3 Nf6 d3 Bc5 Nxe5 d6 Nc4 Bxf2+ Kxf2 Ng...,C42,Petrov's Defense,4,10000000.00,-60


In [45]:
# Let's get rid of those duplicates.
working_data = working_data.drop_duplicates(subset='id')

### Rated Games
This indicates whether a game will affect a user's rating.

In [67]:
working_data.groupby(by='rated').id.count()

rated
False     3646
True     15467
Name: id, dtype: int64

Rated games account for about 81% of all games. What is special about the games that are unrated? Do they feature large skill imbalances between opponents? Are they a time format that a user is not used to playing?

### Number of turns
While there are some rules about draws, there are no limits to the number of turns.

In [70]:
working_data.turns.describe()

count   19113.00
mean       60.51
std        33.49
min         1.00
25%        37.00
50%        55.00
75%        79.00
max       349.00
Name: turns, dtype: float64